In [ ]:
from quartic_sdk import APIClient
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [ ]:
# intialize a Quartic Platform API client  
client = APIClient("https://demo.quartic.ai/",
                   username="aniket@demo.quartic.ai",
                   password="aniketdemo123!")

In [ ]:
assets = client.assets()

In [ ]:
asset = assets.get("name", "CSS AC-169")
asset_tags = client.tags(asset_id=asset.id)

In [ ]:
from datetime import datetime

stop_time = int(datetime.now().timestamp() * 1000) # get current time in millisec unix timestamp
start_time = stop_time - 30 * 60 * 1000 # timestamp 30 mins ago
asset_data_itr = asset.data(start_time=start_time, stop_time=stop_time)

df = pd.DataFrame()
for next_df in asset_data_itr:
    df = pd.concat([df, next_df])
df

In [ ]:
df = df.dropna(axis=1)

In [ ]:
target_col = 15881
feature_cols = [15872,15873,15874,15875,15876]

In [ ]:
df_X = df[feature_cols]
X = df_X.to_numpy()
df_y = df[[target_col]]
y = df_y.to_numpy()

In [ ]:
X_pca = PCA(n_components=2, random_state=123).fit_transform(X)

In [ ]:
reg = LinearRegression().fit(X_pca, y)
preds = reg.predict(X_pca)
preds.flatten()

In [ ]:
from quartic_sdk.model import BaseQuarticModel

class LinearRegressionModel(BaseQuarticModel):
    def __init__(self, name, description, log_level, model):
        self.model = model
        super().__init__(name=name, description=description, log_level=log_level)
        
    def predict(self, input_df: pd.DataFrame):
        self.log.info("Predicting...")
        X = input_df.to_numpy()
        X_pca = PCA(n_components=2, random_state=123).fit_transform(X)
        y_preds = self.model.predict(X_pca)
        return pd.Series(y_preds.flatten())

In [ ]:
lr_pca_model = LinearRegressionModel(name="lr_pca",
                                     description="model trained with data transformed by PCA",
                                     log_level="INFO",
                                     model=reg)

lr_pca_model.save(client=client, output_tag_name='lr_pca', feature_tags=feature_cols, target_tag=target_col, test_df=df_X)